# ALERT SYSTEM

ADX: https://www.investopedia.com/articles/trading/07/adx-trend-indicator.asp

Cron Job: https://crontab.guru/
*/2 * * * *

In [1]:
import pandas as pd
import pandas_ta as ta
import ccxt, yfinance
import requests
import os

In [2]:
exchange = ccxt.binance()

In [3]:
dir = os.getcwd()
path = os.path.join(dir, 'symbols.csv')
symbols = pd.read_csv(path)
symbols = pd.DataFrame(symbols, columns = ["symbol"])
symbols = symbols["symbol"].values.tolist()
symbols

['ETH/USDT', 'BTC/USDT', 'SOL/USDT', 'AVAX/USDT']

In [9]:
for symbol in symbols:
    bars = exchange.fetch_ohlcv(symbol, timeframe = "5m", limit = 500)
    df = pd.DataFrame(bars, columns = ["time", "open", "high", "low", "close", "volume"])
    df['time'] = pd.to_datetime(df['time'], unit = 'ms')    
    
    # technical indicators
    adx = df.ta.adx()
    macd = df.ta.macd(fast = 14, slow = 28)
    rsi = df.ta.rsi()
    
    tenkan=9
    kijun=26
    senkou=52
    ichimoku = ta.ichimoku(df['high'], df['low'], df['close'], tenkan=tenkan, kijun = kijun, senkou = senkou)

    # create composite df
    df = pd.concat([df, adx, macd, rsi, ichimoku[0]], axis = 1)

    # grab the last value
    last_row = df.iloc[-1]
    last_row_ich_kijun = ichimoku[0].iloc[-1]
    print(last_row['ISA_9'])
    
    
    
    cap_time = last_row[0]

    # hook to discord channel
    WEBHOOK_URL = "https://discord.com/api/webhooks/982327105041875004/w_ume6BIxlHe3C9jgRUrUXWSFn3oiLcvpb2118vsNh8iZdRGHxD5LqYDer2cGwsjd_sL"

    # logic
    if last_row["ADX_14"] > 25:
        if (last_row_ich_kijun['ISA_9'] > last_row_ich_kijun['ISB_26']): # future up trend
            if (last_row['close'] > last_row['ISA_9']) & (last_row['close'] > last_row['ISB_26']):
                if (row['ITS_9'] > row['IKS_26']):
                     message = f"{cap_time} STRONG UP TREND: {symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"
        
    if last_row["ADX_14"] > 25:
        if (last_row_ich_kijun['ISA_9'] < last_row_ich_kijun['ISB_26']): # future up trend
            if (last_row['close'] < last_row['ISA_9']) & (last_row['close'] < last_row['ISB_26']):
                if (row['ITS_9'] < row['IKS_26']):
                     message = f"{cap_time} STRONG DOWN TREND: {symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"
           
        payload = {
            "username" : "alertbot",
            "content" : message
        }

        print(message)
        requests.post(WEBHOOK_URL, json = payload)

    if last_row["ADX_14"] < 25:
        message = f"{cap_time} no trend:{symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"

        payload = {
            "username" : "alertbot",
            "content" : message
        }
        
        print(message)
        requests.post(WEBHOOK_URL, json = payload)

1776.2824999999998
2022-06-04 17:40:00 no trend:ETH/USDT: The ADX is  21.66 +DI  19.47 -DI  30.08
29778.045000000002
2022-06-04 17:40:00 no trend:ETH/USDT: The ADX is  21.66 +DI  19.47 -DI  30.08
37.5325
2022-06-04 17:40:00 no trend:SOL/USDT: The ADX is  16.14 +DI  21.05 -DI  22.34
23.262500000000003
2022-06-04 17:40:00 no trend:AVAX/USDT: The ADX is  16.35 +DI  19.88 -DI  22.46
